<a href="https://colab.research.google.com/github/JuanGarza97/Spanish_Nahuatl_Translator/blob/main/Avance5_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AVANCE 5. MODELO FINAL**

Nombre del Trabajo:
AI for Good: Human Heritage
Automatic pre-Hispanic to Spanish Recogniser and Translator


Nombre del asesor del proyecto: Dr. Juan Arturo Nolazco Flores

Equipo: 20
Nombre de alumnos:
 * Juan Carlos Garza Sanchez	A00821522
 * Sinaí Avalos Rivera 			A01730466

## **FEW-SHOTS LEARNING**

Este proyecto tiene como objetivo evaluar y comparar el desempeño de dos modelos de traducción automática, Gemini y Llama3, en la tarea de traducción de oraciones en una lengua indígena a su correspondiente en español.

Luego evalúa esa traducción utilizando las métricas METEOR, BLEU y ROUGE. Se eligieron estos modelos debido a que son de uso gratuito y son ampliamente utilizados para few shots learning.

Este modelo, incluye la comparación y análisis de cómo los shots o el número de ejemplos de referencia proporcionados a los modelos afectan el rendimiento de los modelos, por lo que se realiza un análisis comparativo para identificar qué tan bien cado uno se adapta a la lingüiística.

Los modelos de ensamble son populares en tareas de aprendizaje automático debido a su capacidad para combinar varios modelos base y mejorar el rendimiento general. Sin embargo, en este proyecto específico de traducción automática, se presentan las siguientes razones por las cuales los modelos de ensamble no pueden ser considerados:

* Alta complejidad computacional: Debido a que combinan los resultados de varios modelos base, lo que significa que la carga computacional se incrementa significativamente. Esto no solo aumentaría el tiempo necesario para realizar las inferencias, sino también la memoria y los recursos de procesamiento requeridos.

* Pérdida de interpretabilidad: Uno de los beneficios de trabajar con modelos individuales como Gemini y Llama3 es que se pueden analizar y comprender más fácilmente sus decisiones y desempeño.

* Desempeño no garantizado: Aunque  pueden mejorar el desempeño en algunas tareas, en otros casos, no siempre logran superar el rendimiento de un modelo individual bien entrenado, especialmente en dominios específicos como la traducción automática.

* Limitación en la generalización: Para lenguas poco representadas o complejas como la que se traduce en este proyecto, los modelos base deben tener un alto nivel de conocimiento sobre la lengua y las estructuras lingüísticas involucradas.

* Disponibilidad de modelos base optimizados: En lugar de combinar varios modelos que podrían estar menos especializados en el dominio o la lengua, se puede optar por un modelo base de alto rendimiento, como los utilizados en este proyecto (Gemini y Llama3), que han sido entrenados y afinados para tareas de traducción específica, maximizando así la precisión y efectividad.

In [1]:
!pip install google-cloud-aiplatform nltk==3.8.1 rouge-score

In [2]:
import nltk
import re

# Recursos de NLTK para tokenizacion y WordNet
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

import pandas as pd
import google.generativeai as genai

from huggingface_hub import InferenceClient

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Cargar dataset
df = pd.read_csv("hf://datasets/somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl/train.csv")
df = df[['sp', 'nah']].rename(columns={'sp': 'spanish', 'nah': 'nahuatl'})
df = df.dropna(axis=0, how='any')
df = df.astype(str)
df

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,spanish,nahuatl
0,"Y así, cuando hizo su ofrenda de fuego, se sie...",Auh in ye yuhqui in on tlenamacac niman ye ic ...
1,"¿Si es jade, si es oro, acaso no tendrá que ir...","¿In chalchihuitl, teocuitlatl, mach ah ca on yaz?"
2,"Y cuando el Sol estuvo solo en el cielo, enseg...",Auh yn oyuh in yoca hualmotlalli tonatiuh ylhu...
3,Allá acudieron asimismo los señores del cabild...,Yn oncan mohuicatza yhuan yn ciudad cabildo tl...
4,Usos:,Kualtia:
...,...,...
20023,"El Espíritu y la esposa dicen: ""¡Ven!""El que ...",On Espíritu Santo niman isihuau on Borreguito ...
20024,Yo advierto a todo el que oye las palabras de...,Nemechtlachicahuilia nenmochimej yejhuan nenqu...
20025,y si alguno quita de las palabras del libro d...,Niman tla yacaj quipopolohuilis itemachtil Dio...
20026,"El que da testimonio de estas cosas dice: ""¡S...","On yejhuan quipantlantia yejhua in, ijquin qui..."


In [4]:
# Configurar la API de Gemini
GOOGLE_API_KEY = # Reemplazar con su KEY
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-pro")

In [5]:
# Configurar la API de Hugging Face para Llama3

from huggingface_hub import login
login(token="# Reemplazar con su KEY")


HF_API_KEY = "# Reemplazar con su KEY"
# client = InferenceClient(model="meta-llama/Meta-Llama-3-8B", token=HF_API_KEY)
client = InferenceClient(model="meta-llama/Meta-Llama-3-8B-Instruct", token=HF_API_KEY)

In [18]:
# Crear el prompt con un solo ejemplo
def create_prompt(target_sentence, example):
    return f"""
    Translate the following Nahuatl sentence to Spanish:
    Example:
    "{example['nahuatl'].values[0]}" -> "{example['spanish'].values[0]}"

    Now, translate this sentence:
    "{target_sentence}"
    """

In [19]:
# Funciones de generación
def gemini_generate(prompt):
    response = gemini_model.generate_content(prompt)
    return response.text.strip()

In [20]:
def llama3_generate(prompt):
    return client.text_generation(prompt, max_new_tokens=50).strip()

In [9]:
# Evaluación del modelo
def evaluate_model(model_func, model_name, shots):
    row = df.sample(n=1)
    examples = df.sample(n=shots)
    prompt = create_prompt(row['nahuatl'].values[0], examples)

    prediction = model_func(prompt)
    reference = row['spanish'].values[0]

    # Evaluaciones
    meteor = meteor_score([word_tokenize(reference)], word_tokenize(prediction))
    smoother = SmoothingFunction().method4
    bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(prediction), weights=(0.5, 0.5), smoothing_function=smoother)
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    rouge1 = scorer.score(reference, prediction)['rouge1'].fmeasure

    return model_name, shots, meteor, bleu, rouge1

In [21]:
# Agregar las oraciones originales y traducidas a los resultados
def evaluate_model_with_sentences(model_func, model_name, shots):
    row = df.sample(n=1)
    examples = df.sample(n=shots)
    prompt = create_prompt(row['nahuatl'].values[0], examples)

    prediction = model_func(prompt)
    reference = row['spanish'].values[0]
    original_sentence = row['nahuatl'].values[0]  # Oración original en Nahuatl

    # Evaluaciones
    meteor = meteor_score([word_tokenize(reference)], word_tokenize(prediction))
    smoother = SmoothingFunction().method4
    bleu = sentence_bleu([word_tokenize(reference)], word_tokenize(prediction), weights=(0.5, 0.5), smoothing_function=smoother)
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    rouge1 = scorer.score(reference, prediction)['rouge1'].fmeasure

    return model_name, shots, meteor, bleu, rouge1, original_sentence, prediction

In [22]:
# # Ejecutar pruebas con distintos shots
shots_values = [1, 3, 5]
models = [(gemini_generate, "Gemini"), (llama3_generate, "Llama3")]

# Ejecutar pruebas con distintos shots y agregar oraciones originales y traducidas
results_with_sentences = []
for model_func, model_name in models:
    for shots in shots_values:
        results_with_sentences.append(evaluate_model_with_sentences(model_func, model_name, shots))

In [24]:
# Crear DataFrame con las columnas adicionales de oraciones originales y traducidas
df_results = pd.DataFrame(results_with_sentences, columns=["Modelo", "Shots", "METEOR", "BLEU", "ROUGE-1", "Oración Original", "Traducción"])
print(df_results)

   Modelo  Shots    METEOR      BLEU   ROUGE-1  \
0  Gemini      1  0.222754  0.124541  0.237288   
1  Gemini      3  0.199352  0.145138  0.313514   
2  Gemini      5  0.316832  0.183494  0.309278   
3  Llama3      1  0.000000  0.000000  0.000000   
4  Llama3      3  0.070423  0.021507  0.000000   
5  Llama3      5  0.170375  0.061859  0.207792   

                                    Oración Original  \
0  Yejhuamej on yejhuan nemij quen tlanamactin, m...   
1  36.\tAhmo tle ye tihualmoxicoz, immicantla, im...   
2  Auh quicacque yn Huitzillihuitzin yhuan yn Qua...   
3  Sa no tejhuamej oticaquej on tlajtojli yejhuan...   
4  Nima ye yc micalli, yc unca much tlamaque y me...   
5  Yhuan cenca miec yn apachiuh yn acallac yn inc...   

                                          Traducción  
0  Y ellos vivieron como sabios, pero que no tuvi...  
1  36.\tNo hay nada en que nos podamos confiar, e...  
2  Y dieron a Huitzilihuitl y a Cuatlecohuatzin y...  
3  -> "¿Qué es lo que se llama Teco?

**EVALUACIÓN DE RESULTADOS:**

Para evaluar estos resultados, vamos a analizar las métricas METEOR, BLEU, y ROUGE-1, y lo que cada una de ellas refleja en el contexto de las oraciones originales y traducidas:

1. METEOR (Metric for Evaluation of Translation with Explicit ORdering):
La METEOR mide la calidad de las traducciones considerando sinónimos y diferentes órdenes de las palabras.
Valores altos indican una mejor correspondencia semántica entre la traducción y la referencia.

* Gemini muestra un aumento en la puntuación de METEOR al aumentar los shots: de 0.22 con 1 shot a 0.32 con 5 shots. Esto indica que Gemini logra una mayor correspondencia semántica a medida que se le da más contexto.

* Mientras que Llama3 tiene puntuaciones bajas en METEOR. A pesar de que mejora con más shots (de 0 con 1 shot a 0.17 con 5 shots), sigue siendo relativamente baja, lo que sugiere que Llama3 no está capturando bien el significado durante la traducción.

2. BLEU (Bilingual Evaluation Understudy):
Mide la precisión de n-gramas, enfocándose en qué tan bien las palabras de la traducción coinciden con las de la referencia.
Valores altos indican que la traducción tiene un gran solapamiento en términos de secuencia de palabras.

* Gemini también muestra un aumento en BLEU con más shots: de 0.12 con 1 shot a 0.18 con 5 shots. Sin embargo, sus valores siguen siendo relativamente bajos, lo que sugiere que la traducción es algo diferente de la referencia, pero con algo de coincidencia.

* Llama3, al igual que con METEOR, muestra un bajo desempeño en BLEU, con valores cercanos a 0. Esto sugiere que la traducción generada no es muy precisa en términos de las secuencias de palabras.

3. ROUGE-1 (Recall-Oriented Understudy for Gisting Evaluation):
ROUGE-1 mide la recuperación de n-gramas unigram, lo que indica qué tan bien las palabras clave de la traducción coinciden con las de la referencia.
Valores altos indican que la traducción es capaz de recuperar correctamente las palabras clave de la referencia.

* Gemini muestra mejoras con más shots, pasando de 0.24 con 1 shot a 0.31 con 3 shots, y luego se estabiliza con 5 shots. Esto indica que Gemini es consistente en la recuperación de palabras clave, con un rendimiento ligeramente mejor con más contexto.

* Llama3, por otro lado, tiene un puntaje bajo de ROUGE-1 en todos los casos, con una mejora pequeña de 0.00 con 1 shot a 0.21 con 5 shots. Aunque mejora ligeramente, aún no logra una buena recuperación de palabras clave de la referencia.